In [9]:
__author__ = 'Nicolas Riera'

In [10]:
import pandas as pd
from scipy.stats import chisquare , norm
from numpy import sqrt
import csv

In [11]:
reporte = "reporte.csv"
datos = pd.read_csv(reporte, sep=';')
lista_transiciones = list(zip(datos['transicion'], datos["cluster"]))
# creamos el diccionario
cuenta_transicion = {}
for i in lista_transiciones:
    if not i[0] in cuenta_transicion:
        cuenta_transicion[i[0]] = {}
    if not i[1] in cuenta_transicion[i[0]]:
        cuenta_transicion[i[0]][i[1]] = 0
    cuenta_transicion[i[0]][i[1]] += 1
                           
lista_final = []

for i in cuenta_transicion:
    lista ={}
    lista_nombres = list(cuenta_transicion[i].keys())
    for j in lista_nombres:
        lista[j] = cuenta_transicion[i][j]
    lista["Transicion"] = i 
    lista_final.append(lista)


df = pd.DataFrame(lista_final)
df

,Altamente Descompensado,Compensado,Medianamente Descompensado,Mejora,Transicion
0,155.0,116,90.0,26.0,Casual
1,NaN,1,NaN,NaN,Der. Ciclica
2,119.0,115,65.0,30.0,Interr
3,183.0,182,80.0,38.0,Delegador
4,2.0,1,2.0,NaN,Particip. Coord
5,33.0,23,9.0,6.0,Participativo
6,NaN,1,1.0,NaN,Contraderivacion
7,259.0,171,103.0,62.0,Deleg. reasig.


In [12]:
# obtener el valor_p

lista = [list(x) for x in zip(df["Compensado"], df["Mejora"], df["Medianamente Descompensado"], 
                              df["Altamente Descompensado"], df["Transicion"])]
lista_datos = [x[:4] for x in lista]

transiciones_p = []
lista_nombre = ["Compensado", "Mejora", "Medianamente Descompensado", "Altamente Descompensado"]

def p_recursivo(transiciones_p, lista_datos, lista, j , lista_nombre):
     
    for i in range(len(lista_datos)-1):
        transicion = lista[0][-1] + " v/s " + lista[i+1][-1]
        prop1 = lista_datos[0][j]/sum(lista_datos[0])
        prop2 = lista_datos[i+1][j]/sum(lista_datos[i+1])
        Z = (prop1 - prop2)/(sqrt((prop1*(1-prop1)/sum(lista_datos[0])))+sqrt(prop2*(1-prop2)/sum(lista_datos[i+1])))
        p_value = norm.sf(abs(Z))
        transiciones_p.append([transicion, lista_nombre[j], p_value])
    
    if len(lista_datos)>0:
        lista_datos.pop(0)
        lista.pop(0)
        return p_recursivo(transiciones_p, lista_datos, lista, j , lista_nombre)
    else:
        return transiciones_p

lista_p = []
for j in range(len(lista_nombre)):
    aux2 = lista_datos.copy()
    aux3 = lista.copy()
    transiciones_p = []
    lista_p += p_recursivo(transiciones_p, aux2, aux3, j , lista_nombre)
nueva = []
for p in lista_p:
    if not p[2] != p[2]:
        nueva.append(p)
lista_p = nueva

In [13]:
def write_file(delimiter, lista_p, name):
    count = 0
    rows = lista_p
    report = open('{0}.csv'.format(name), 'w', newline='')
    wr = csv.writer(report, delimiter=delimiter)
    header = ["Transiciones", "Estado Transicion", "valor_p"]
    wr.writerow(header)
    for i in rows:
        wr.writerow(i)
        count += 1
    report.close()
    
write_file(";", lista_p, "valores_p_transiciones")


In [14]:
datos = pd.read_csv('valores_p_transiciones.csv', sep=';')
datos

,Transiciones,Estado Transicion,valor_p
0,Casual v/s Interr,Compensado,0.157558
1,Casual v/s Delegador,Compensado,0.044576
2,Casual v/s Participativo,Compensado,0.379412
3,Casual v/s Deleg. reasig.,Compensado,0.383966
4,Interr v/s Delegador,Compensado,0.286342
5,Interr v/s Participativo,Compensado,0.377194
6,Interr v/s Deleg. reasig.,Compensado,0.082875
7,Delegador v/s Participativo,Compensado,0.247812
8,Delegador v/s Deleg. reasig.,Compensado,0.013823
9,Participativo v/s Deleg. reasig.,Compensado,0.310903


In [15]:
# filtro por significancia del 5%

datos_filtro = sorted(list(filter(lambda x: x[2] < 0.05, lista_p)), key = lambda x : x[2])
write_file(";", datos_filtro, "transiciones_rechazadas")


In [16]:
datos = pd.read_csv('transiciones_rechazadas.csv', sep=';')
datos

,Transiciones,Estado Transicion,valor_p
0,Delegador v/s Deleg. reasig.,Compensado,0.013823
1,Casual v/s Delegador,Medianamente Descompensado,0.040638
2,Casual v/s Participativo,Medianamente Descompensado,0.041323
3,Casual v/s Delegador,Compensado,0.044576
